In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import csv
%load_ext sql 

In [2]:
con = sql.connect("my_data1.db")
cur = con.cursor()

%sql sqlite:///my_data1.db

In [3]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"
df = pd.read_csv(url)
df.head()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [4]:
df.to_sql('SPACEXTBL', con, if_exists='replace', index=False, method='multi')

101

In [5]:
%%sql
create table SPACEX_TABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEX_TABLE already exists
[SQL: create table SPACEX_TABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [10]:
%%sql
select distinct Launch_Site from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [11]:
%%sql
select distinct Launch_Site from SPACEXTBL
where Launch_Site like 'CCA%'

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS SLC-40


In [40]:
%%sql

select AVG(PAYLOAD_MASS__KG_) as Avg_Payload_Mass from SPACEXTBL
where Booster_Version like 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


Avg_Payload_Mass
2928.4


In [51]:
%%sql

select * from SPACEXTBL
where date = (
    select min(Date) as First_Attempt_Date from SPACEXTBL
    where Landing_Outcome like 'Success%pad%'
)


 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2015-12-22,1:29:00,F9 FT B1019,CCAFS LC-40,OG2 Mission 2 11 Orbcomm-OG2 satellites,2034,LEO,Orbcomm,Success,Success (ground pad)


In [61]:
%%sql

select Booster_Version from SPACEXTBL
where Landing_Outcome like 'Success%ship%' 
and PAYLOAD_MASS__KG_ > 4000
and PAYLOAD_MASS__KG_ < 6000

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [66]:
%%sql

select count(*) as Number_Successes from SPACEXTBL
where Mission_Outcome like 'Success%'

 * sqlite:///my_data1.db
Done.


Number_Successes
100


In [72]:
%%sql

select count(*) as Number_Failures from SPACEXTBL
where Mission_Outcome not like 'Success%'

 * sqlite:///my_data1.db
Done.


Number_Failures
1


In [84]:
%%sql

select Booster_Version from SPACEXTBL
where PAYLOAD_MASS__KG_ = (
    select max(PAYLOAD_MASS__KG_) from SPACEXTBL
)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [91]:
%%sql

select Date, substr(Date, 6, 2) as Month, Booster_Version, Launch_Site, Landing_Outcome from SPACEXTBL
where Date like '%2015%'
and Landing_Outcome like '%drone%'

 * sqlite:///my_data1.db
Done.


Date,Month,Booster_Version,Launch_Site,Landing_Outcome
2015-01-10,01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
2015-04-14,04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)
2015-06-28,06,F9 v1.1 B1018,CCAFS LC-40,Precluded (drone ship)


In [ ]:
 select Date, 